In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [3]:
df = pd.read_excel("Population by Locale.xlsx")
df

,Year,Urban population,Rural population
0,1960,136944.0,1267832
1,1961,144898.0,1293546
2,1962,153405.0,1320122
3,1963,162465.0,1347492
4,1964,172077.0,1375541
...,...,...,...
66,2026,2796265.0,3389751
67,2027,2896081.0,3420401
68,2028,2998273.0,3450178
69,2029,3102932.0,3479197


In [4]:
df["Year"]

0     1960
1     1961
2     1962
3     1963
4     1964
      ... 
66    2026
67    2027
68    2028
69    2029
70    2030
Name: Year, Length: 71, dtype: int64

In [5]:
fig = make_subplots(rows = 1, cols = 1)
fig.add_trace(go.Scatter(x = df["Year"], y = df["Urban population"], marker = dict(color = "Red"), name = "Urban Population"))
fig.add_trace(go.Scatter(x = df["Year"], y = df["Rural population"], marker = dict(color = "Blue"), name = "Rural Population"))
fig.show()

In [34]:
from scipy.optimize import curve_fit

def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

p0 = [float(max(df["Rural population"])), float(np.median(df["Year"])),1,float(min(df["Rural population"]))] # this is an mandatory initial guess
popt, pcov = curve_fit(sigmoid, df["Year"].astype("float"), df["Rural population"].astype("float"),p0, method='lm', maxfev=10000)

In [35]:
p1 = [float(max(df["Urban population"])), float(np.median(df["Year"])),1,float(min(df["Urban population"]))] # this is an mandatory initial guess
popt1, pcov1 = curve_fit(sigmoid, df["Year"].astype("float"), df["Urban population"].astype("float"),p0, method='lm', maxfev=10000)

In [36]:
popt1

array([6.04567007e+06, 2.02901066e+03, 6.09801647e-02, 5.79715866e+04])

In [37]:
popt

array([ 2.56985358e+09,  1.17200172e+03,  7.88456541e-03, -2.56347165e+09])

In [39]:
fig = make_subplots(rows = 1, cols = 1)

fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt), line = {"dash": "dot"}, marker = dict(color = "Blue"), name = "Predicted Rural Population"))
fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt1), line = {"dash": "dot"}, marker = dict(color = "Green"), name = "Predicted Urban Population"))
fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt)+sigmoid([i for i in range(1960, 2041)], *popt1), line = {"dash": "dot"}, marker = dict(color = "Red"), name = "Predicted Total Population"))

fig.add_trace(go.Scatter(x = df["Year"][:64], y = df["Rural population"][:64], marker = dict(color = "Blue"), name = "Rural Population"))
fig.add_trace(go.Scatter(x = df["Year"][:64], y = df["Urban population"][:64], marker = dict(color = "Green"), name = "Urban Population"))
fig.add_trace(go.Scatter(x = df["Year"][:64], y = df["Rural population"][:64]+df["Urban population"][:64], marker = dict(color = "Red"), name = "Total Population"))

fig.update_xaxes(title = "Year")
fig.update_yaxes(title = "Population")

fig.update_layout(title = "Eritrea Population Model", width = 750)

fig.update_traces(patch={"line": {"width": 4, "dash": 'dot'}}, selector={"legendgroup": "Rural Population Pred"}) 

fig.update_layout(
    plot_bgcolor='white'
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.show()

In [59]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt), line = {"dash": "dot"}, marker = dict(color = "Blue"), name = "Predicted Rural Population"))
fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt1), line = {"dash": "dot"}, marker = dict(color = "Green"), name = "Predicted Urban Population"))
fig.add_trace(go.Scatter(x = [i for i in range(1960, 2041)], y = sigmoid([i for i in range(1960, 2041)], *popt)/(sigmoid([i for i in range(1960, 2041)], *popt) + sigmoid([i for i in range(1960, 2041)], *popt1)), marker = dict(color = "Orange"), name = "Predicted Total Population"), secondary_y=True)

fig.add_trace(go.Scatter(x = df["Year"][:64], y = df["Rural population"][:64], marker = dict(color = "Blue"), name = "Rural Population"))
fig.add_trace(go.Scatter(x = df["Year"][:64], y = df["Urban population"][:64], marker = dict(color = "Green"), name = "Urban Population"))

fig.update_xaxes(title = "Year")
fig.update_yaxes(title = "Population")

fig.update_yaxes(title_text="Rural to Total", range=[0, 1], secondary_y=True)

fig.update_layout(title = "Eritrea Population Model", width = 1000)

fig.update_traces(patch={"line": {"width": 4, "dash": 'dot'}}, selector={"legendgroup": "Rural Population Pred"}) 

fig.update_layout(
    plot_bgcolor='white'
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

fig.update_layout(showlegend = False)

fig.show()